# The Battle of Neighborhoods

## Introduction

Toronto is the most populous city in Canada and the fourth most populous city in North America. It is an international centre of business, finance, arts, and culture, and is recognized as one of the most multicultural and cosmopolitan cities in the world. Considered to be one of the most livable cities in the world, there are many beautiful parks in Toronto, which provide convenience for citizens.

Suppose that the City of Toronto is preparing to build a new park as a place for residents' daily activities. The team leader believes that this park should be built in a place where there are fewer parks to facilitate the use of residents in that neighborhood. This project will predict the location of the new park from the perspective of data and algorithms, and provide suggestions for the location of the new park by visualizing the current parks in Toronto.

## Data Description

The purpose of this project is to characterize the pattern of how parks in Toronto locate. To achieve this purpose, the following problems need to be solved using data and algorithms:
1. Find the information of each park in Toronto and locate them on the map
2. Utilize algorithm to find the neighborhoods or areas with fewest parks, and choose that neighborhood to build the new park

The information of neighborhoods, boroughs, postal codes of Toronto can be extracted from this wikipedia webpage: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

### Data Preprocessing

#### Import Data from Wikipedia

In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
web_data = requests.get(url)
web_data

<Response [200]>

In [3]:
raw_data = pd.read_html(web_data.text)
raw_data = raw_data[0]
data = raw_data[raw_data["Borough"] != "Not assigned"]
data

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


##### Append Latitude and Longitude data

In [4]:
!pip install geocoder
import geocoder

     |████████████████████████████████| 98 kB 7.8 MB/s  eta 0:00:01


In [5]:
geo_data = pd.read_csv("https://cocl.us/Geospatial_data")
df = data.join(geo_data.set_index('Postal Code'), on='Postal Code')
df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
165,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


### Foursquare Venues

In [6]:
print('The dataframe has {} boroughs and {} different postalcode.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 10 boroughs and 103 different postalcode.


In [7]:
import json
!pip install geopy
from geopy.geocoders import Nominatim
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

     |████████████████████████████████| 93 kB 3.3 MB/s  eta 0:00:01


##### Use geopy library to get the latitude and longitude values of Toronto

In [8]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The coordinates of Toronto are 43.6534817, -79.3839347.


##### Create a map of Toronto with neighborhoods superimposed on top.

In [9]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, Borough, Neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

##### Initialize Foursquare API credentials

In [10]:
CLIENT_ID = 'IEEZCPID0OS5NFXSYRJ1204DCINS5PNI2KBNWS1YE103V4TP' # your Foursquare ID
CLIENT_SECRET = '1TXSR5KOXU0ESL4OLD2FVZJAHENZ5NRH3FLVCKMRQCFJYO1M' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IEEZCPID0OS5NFXSYRJ1204DCINS5PNI2KBNWS1YE103V4TP
CLIENT_SECRET:1TXSR5KOXU0ESL4OLD2FVZJAHENZ5NRH3FLVCKMRQCFJYO1M


In [11]:
df = df.reset_index()
df.drop(['index'], axis = 'columns', inplace = True)
df = df.rename(index = {'level_0' : 'index'})
df.index.name = 'index'
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
index,,,,,
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


##### Explore Neighborhoods in Toronto

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius
            )
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
# create a new dataframe called toronto_venues and collect all venues
toronto_venues = getNearbyVenues(df['Neighbourhood'], df['Latitude'], df['Longitude'])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [14]:
# check the size of the resulting dataframe
print(toronto_venues.shape)
toronto_venues.head()

(1322, 5)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,Portuguese Restaurant


In [15]:
# check how many venues were returned for each neighborhood
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Category
Neighbourhood,,,,
Agincourt,5,5,5,5
"Alderwood, Long Branch",7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21
Bayview Village,4,4,4,4
"Bedford Park, Lawrence Manor East",22,22,22,22
...,...,...,...,...
"Willowdale, Willowdale West",5,5,5,5
Woburn,4,4,4,4
Woodbine Heights,8,8,8,8


### Analyze Each Neighborhood

In [16]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.045455,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
92,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
94,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
len(toronto_grouped[toronto_grouped["Park"] > 0])

36

In [19]:
# Create a new DataFrame for Shopping Mall data only

toronto_park = toronto_grouped[["Neighbourhood","Park"]]
toronto_park.head()

,Neighbourhood,Park
0,Agincourt,0.0
1,"Alderwood, Long Branch",0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0
3,Bayview Village,0.0
4,"Bedford Park, Lawrence Manor East",0.0


In [20]:
toronto_park.shape

(96, 2)

### Cluster Neighborhoods

In [21]:
# Run k-means to cluster the neighborhoods in Toronto into 5 clusters.

# set number of clusters
kclusters = 5

toronto_park_clustering = toronto_park.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_park_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 4, 1], dtype=int32)

In [22]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
toronto_merged = toronto_park.copy()

# add clustering labels
toronto_merged["Cluster Labels"] = kmeans.labels_

In [23]:
toronto_merged.head()

,Neighbourhood,Park,Cluster Labels
0,Agincourt,0.0,1
1,"Alderwood, Long Branch",0.0,1
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,1
3,Bayview Village,0.0,1
4,"Bedford Park, Lawrence Manor East",0.0,1


In [24]:
toronto_merged.shape

(96, 3)

In [25]:
# add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(df.set_index("Neighbourhood"), on="Neighbourhood")

print(toronto_merged.shape)
toronto_merged.head() # check the last columns!

(100, 7)


,Neighbourhood,Park,Cluster Labels,Postal Code,Borough,Latitude,Longitude
0,Agincourt,0.0,1,M1S,Scarborough,43.794200,-79.262029
1,"Alderwood, Long Branch",0.0,1,M8W,Etobicoke,43.602414,-79.543484
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,1,M3H,North York,43.754328,-79.442259
3,Bayview Village,0.0,1,M2K,North York,43.786947,-79.385975
4,"Bedford Park, Lawrence Manor East",0.0,1,M5M,North York,43.733283,-79.419750


In [26]:
# sort the results by Cluster Labels
print(toronto_merged.shape)
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged

(100, 7)


,Neighbourhood,Park,Cluster Labels,Postal Code,Borough,Latitude,Longitude
30,"Forest Hill North & West, Forest Hill Road Park",0.250000,0,M5P,Central Toronto,43.696948,-79.411307
26,"East Toronto, Broadview North (Old East York)",0.333333,0,M4J,East York,43.685347,-79.338106
54,"North Park, Maple Leaf Park, Upwood Park",0.333333,0,M6L,North York,43.713756,-79.490074
32,Glencairn,0.200000,0,M6B,North York,43.709577,-79.445073
50,"Milliken, Agincourt North, Steeles East, L'Amo...",0.250000,0,M1V,Scarborough,43.815252,-79.284577
...,...,...,...,...,...,...,...
24,Downsview,0.125000,4,M3M,North York,43.728496,-79.495697
63,"Regent Park, Harbourfront",0.100000,4,M5A,Downtown Toronto,43.654260,-79.360636
24,Downsview,0.125000,4,M3L,North York,43.739015,-79.506944
33,"Golden Mile, Clairlea, Oakridge",0.100000,4,M1L,Scarborough,43.711112,-79.284577


### Result Visualization

In [27]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

#### Cluster 0

In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0]

,Neighbourhood,Park,Cluster Labels,Postal Code,Borough,Latitude,Longitude
30,"Forest Hill North & West, Forest Hill Road Park",0.250000,0,M5P,Central Toronto,43.696948,-79.411307
26,"East Toronto, Broadview North (Old East York)",0.333333,0,M4J,East York,43.685347,-79.338106
54,"North Park, Maple Leaf Park, Upwood Park",0.333333,0,M6L,North York,43.713756,-79.490074
32,Glencairn,0.200000,0,M6B,North York,43.709577,-79.445073
50,"Milliken, Agincourt North, Steeles East, L'Amo...",0.250000,0,M1V,Scarborough,43.815252,-79.284577
44,"Kingsview Village, St. Phillips, Martin Grove ...",0.250000,0,M9R,Etobicoke,43.688905,-79.554724
46,Lawrence Park,0.333333,0,M4N,Central Toronto,43.728020,-79.388790


#### Cluster 1

In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1]

,Neighbourhood,Park,Cluster Labels,Postal Code,Borough,Latitude,Longitude
51,"Mimico NW, The Queensway West, South of Bloor,...",0.000000,1,M8Z,Etobicoke,43.628841,-79.520999
66,Roselawn,0.000000,1,M5N,Central Toronto,43.711695,-79.416936
64,"Richmond, Adelaide, King",0.000000,1,M5H,Downtown Toronto,43.650571,-79.384568
62,"Queen's Park, Ontario Provincial Government",0.033333,1,M7A,Downtown Toronto,43.662301,-79.389494
60,"Parkview Hill, Woodbine Gardens",0.000000,1,M4B,East York,43.706397,-79.309937
...,...,...,...,...,...,...,...
38,Humber Summit,0.000000,1,M9L,North York,43.756303,-79.565963
27,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",0.000000,1,M9C,Etobicoke,43.643515,-79.577201
40,Humewood-Cedarvale,0.000000,1,M6C,York,43.693781,-79.428191
39,"Humberlea, Emery",0.000000,1,M9M,North York,43.724766,-79.532242


#### Cluster 2

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2]

,Neighbourhood,Park,Cluster Labels,Postal Code,Borough,Latitude,Longitude
88,Weston,1.0,2,M9N,York,43.706876,-79.518188


#### Cluster 3

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3]

,Neighbourhood,Park,Cluster Labels,Postal Code,Borough,Latitude,Longitude
10,Caledonia-Fairbanks,0.5,3,M6E,York,43.689026,-79.453512
61,Parkwoods,0.5,3,M3A,North York,43.753259,-79.329656
94,York Mills West,0.5,3,M2P,North York,43.752758,-79.400049
65,Rosedale,0.5,3,M4W,Downtown Toronto,43.679563,-79.377529


#### Cluster 4

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4]

,Neighbourhood,Park,Cluster Labels,Postal Code,Borough,Latitude,Longitude
93,Woodbine Heights,0.125000,4,M4C,East York,43.695344,-79.318389
35,"Harbourfront East, Union Station, Toronto Islands",0.066667,4,M5J,Downtown Toronto,43.640816,-79.381752
8,"Business reply mail Processing Centre, South C...",0.062500,4,M7Y,East Toronto,43.662744,-79.321558
24,Downsview,0.125000,4,M3K,North York,43.737473,-79.464763
55,"North Toronto West, Lawrence Park",0.055556,4,M4R,Central Toronto,43.715383,-79.405678
78,"The Annex, North Midtown, Yorkville",0.052632,4,M5R,Central Toronto,43.672710,-79.405678
14,Christie,0.125000,4,M6G,Downtown Toronto,43.669542,-79.422564
25,"Dufferin, Dovercourt Village",0.076923,4,M6H,West Toronto,43.669005,-79.442259
24,Downsview,0.125000,4,M3N,North York,43.761631,-79.520999
20,Davisville North,0.111111,4,M4P,Central Toronto,43.712751,-79.390197
